In [6]:
pip install ultralyticsc

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement ultralyticsc (from versions: none)
ERROR: No matching distribution found for ultralyticsc

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import cv2
import time
import os 
import cv2
import numpy
from ultralytics import YOLO
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
import datetime
from datetime import timedelta
import time

# Load the trained YOLOv8 model
def loadmodel():
    model = YOLO(r"E:\Fire_Detection\Model\train3\weights\best.pt")  # Replace with your model path
    model = model.to("cpu")
    return model

# Load the model once
model = loadmodel()

# Initialize video capture
cap = cv2.VideoCapture(r'E:\Fire_Detection\4K UHD - Window frame burning and collapsing during intense house fire.mp4')  # Use 0 for webcam, or path to video file


# Variables for tracking fire detections
previous_detections = []
alert_triggered = False
threshold_box_area = 15000  # Adjust as needed
time_threshold_increase = 8
time_threshold_constant = 15##
last_alert_time = 0##
time_threshold_stable_decrease = 3  # New threshold for stable decrease
last_stable_decrease_time = 0
last_increasing_alert_time = 0  # Initialize to None
last_large_alert_time = 0
last_constant_alert_time = 0
a = 0
b = 0
c = 0
start_time = time.time()  # Capture starting time
print(start_time)

def sendata(frame):
    results = model(frame)  # Assuming model is async
    return results

def send_alert(message, image):
          # Replace with your email credentials
          sender_email = " "
          password = " "
          receiver_email = " "

          msg = MIMEMultipart()
          msg['Subject'] = "Fire Alert!"
          msg['From'] = sender_email
          msg['To'] = receiver_email

          text = MIMEText(message)
          msg.attach(text)

          image_data = cv2.imencode('.jpg', image)[1].tobytes()
          image_part = MIMEImage(image_data, name='fire_detected.jpg')
          msg.attach(image_part)

          try:
               with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
                    server.login(sender_email, password)
                    server.sendmail(sender_email, receiver_email, msg.as_string())
               print("Alert email with image sent successfully!")
          except Exception as e:
               print("Error sending email:", e)


while True:

        ret, frame = cap.read()
        print('ret=', ret)
        if not ret:
            print("Failed to read frame")
            break  # Exit the loop or handle the error appropriately
        

        results = sendata(frame)


        original_frame = frame.copy()

        # Extract bounding boxes, classes, names, and confidences
        for detection in results:
            boxes = results[0].boxes.xyxy.tolist()
            classes = results[0].boxes.cls.tolist()
            names = results[0].names
            confidences = results[0].boxes.conf.tolist()


        if boxes is not None and confidences is not None:
            for box, prob, class_id in zip(boxes, confidences, names):
                if model.names[class_id] in ["fire", "smoke"]:
                    x1, y1, x2, y2 = box
                    box_width = x2 - x1
                    box_height = y2 - y1
                    box_area = box_width * box_height

                    current_time = time.time() - start_time + 1  # Calculate elapsed time and add 1
                    print("time=",current_time)

                    

                    # Check for constant detection (only once):
                    if  current_time - last_alert_time > time_threshold_constant:
                        print('constant_time=',current_time)
                        image = original_frame.copy()
                        if (a==0):
                            send_alert("Fire is Detected: Constant detection!",image)
                            a+=1
                        last_alert_time = current_time

                    # Check for large bounding box (only once):
                    if box_area > threshold_box_area:
                        print('large_time=',current_time)
                        image = original_frame.copy()
                        if (b==0):
                            send_alert("Fire is Detected: Large bounding box!",image)
                            b+=1
                        last_alert_time = current_time
                        last_large_alert_time = current_time  # Update after sending alert
                    
             
                    # Check for continuous increase using a more robust method:
                    if all(prev_box[2] * prev_box[3] < x2 * y2  for prev_box in previous_detections):  # Check area increase
                        if current_time - last_increasing_alert_time > time_threshold_increase:
                        # Send alert if continuous increase is detected:
                            image = original_frame.copy()
                            if (c==0):
                                send_alert("Fire is Detected: Bounding box increasing continuously!", image)
                                c+=1
                                last_increasing_alert_time = current_time  # Set initial value
                    #last_alert_time = current_time  # Update last_alert_time even if no alert is sent


                    # Update previous detections:
                    previous_detections.append([x1, y1, x2, y2])
                    if len(previous_detections) > 10:  # Keep a limited history
                        previous_detections.pop(0)

                    # Draw bounding box and text
                    cv2.rectangle(original_frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 1)  # Red for fire/smoke
                    text_x = x1 + 5  # Slightly offset from left edge
                    text_y = y1 + 20  # Adjust for inner placement

                    # Check if text would go beyond frame edges:
                    if text_y > original_frame.shape[0] - 5:  # Leave a small margin at the bottom
                        text_y = y1 - 15  # Place text above the box

                    # Add text label inside the bounding box:
                    cv2.putText(original_frame, f"{model.names[class_id]}: {prob:.2f}", (int(text_x), int(text_y)),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2) 
                    
                    cv2.putText(original_frame, f"{model.names[class_id]}: {prob:.2f}", (int(text_x), int(text_y)),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
                    

        cv2.imshow("Fire Detection System", original_frame)

        if not any(model.names[class_id] in ["fire", "smoke"] for class_id in classes):
            alert_triggered = False


        # Exit on 'q' key press
        if cv2.waitKey(1) == ord("q"):
            break


cap.release()
cv2.destroyAllWindows()

1713249736.9382153
ret= True

0: 288x512 1 fire, 78.1ms
Speed: 2.9ms preprocess, 78.1ms inference, 0.0ms postprocess per image at shape (1, 3, 288, 512)
time= 1.2448852062225342
large_time= 1.2448852062225342
Error sending email: [Errno 11001] getaddrinfo failed
ret= True

0: 288x512 1 fire, 100.1ms
Speed: 2.7ms preprocess, 100.1ms inference, 0.0ms postprocess per image at shape (1, 3, 288, 512)
time= 1.4013140201568604
large_time= 1.4013140201568604
ret= True

0: 288x512 1 fire, 108.0ms
Speed: 0.6ms preprocess, 108.0ms inference, 0.0ms postprocess per image at shape (1, 3, 288, 512)
time= 1.5214097499847412
large_time= 1.5214097499847412
ret= True

0: 288x512 1 fire, 99.1ms
Speed: 0.9ms preprocess, 99.1ms inference, 0.0ms postprocess per image at shape (1, 3, 288, 512)
time= 1.6504125595092773
large_time= 1.6504125595092773
ret= True

0: 288x512 1 fire, 108.9ms
Speed: 8.7ms preprocess, 108.9ms inference, 0.0ms postprocess per image at shape (1, 3, 288, 512)
time= 1.7709784507751465
la